# 📊 E-Commerce Internet Sales Deep Analysis with Visualizations

This notebook explores Internet Sales data and generates **trends and insights** through advanced data visualizations:
- Customer Insights (Pareto, Gender, City)
- Product Insights (Treemap, Top 10 Product Trends)
- Time-Series (Rolling Average, Sales vs Budget)
- RFM Segmentation
- Correlation Heatmaps


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import squarify
import datetime as dt

# Load datasets
calendar = pd.read_csv('../data/DIM_Calendar.csv')
customer = pd.read_csv('../data/DIM_Customer.csv')
product = pd.read_csv('../data/DIM_Product.csv')
sales = pd.read_csv('../data/FACT_InternetSales.csv')
budget = pd.read_excel('../data/Sent Over Data - SalesBudget.xlsx')

# Merge
df = sales.merge(customer, on='CustomerKey').merge(product, on='ProductKey').merge(calendar, left_on='OrderDateKey', right_on='DateKey')
df['OrderDate'] = pd.to_datetime(df['Date'])
df.head()

## 👤 Customer Insights

In [ ]:
# Pareto Analysis (80/20 rule)
customer_sales = df.groupby('FullName')['SalesAmount'].sum().sort_values(ascending=False).reset_index()
customer_sales['Cumulative %'] = customer_sales['SalesAmount'].cumsum() / customer_sales['SalesAmount'].sum() * 100

fig, ax1 = plt.subplots(figsize=(10,6))
sns.barplot(x=customer_sales.index, y='SalesAmount', data=customer_sales, ax=ax1, color='skyblue')
ax2 = ax1.twinx()
sns.lineplot(x=customer_sales.index, y='Cumulative %', data=customer_sales, ax=ax2, color='red', marker='o')
ax1.set_title('Pareto Analysis of Customers')
ax1.set_ylabel('Sales Amount')
ax2.set_ylabel('Cumulative %')
plt.show()

# Sales by Gender
sns.barplot(x='Gender', y='SalesAmount', data=df, estimator=sum, ci=None)
plt.title('Sales by Gender')
plt.show()

# Top 10 Cities by Sales
sales_by_city = df.groupby('Customer City')['SalesAmount'].sum().sort_values(ascending=False).head(10)
sales_by_city.plot(kind='barh', figsize=(10,5), title='Top 10 Cities by Sales')
plt.show()

## 📦 Product Insights

In [ ]:
# Treemap: Sales by Product Category
sales_by_category = df.groupby('Product Category')['SalesAmount'].sum().reset_index()
plt.figure(figsize=(10,6))
squarify.plot(sizes=sales_by_category['SalesAmount'], label=sales_by_category['Product Category'], alpha=.8)
plt.title('Sales Distribution by Product Category')
plt.axis('off')
plt.show()

# Top 10 Products Trend Over Time
top_products = df.groupby('Product Name')['SalesAmount'].sum().nlargest(10).index
trend_top_products = df[df['Product Name'].isin(top_products)].groupby(['Year','MonthNo','Product Name'])['SalesAmount'].sum().reset_index()

plt.figure(figsize=(12,6))
sns.lineplot(x='MonthNo', y='SalesAmount', hue='Product Name', data=trend_top_products, marker='o')
plt.title('Monthly Sales Trend of Top 10 Products')
plt.show()

## 📈 Time-Series Trends

In [ ]:
# Monthly Sales vs Budget (Area Chart)
monthly_sales = df.groupby(['Year','MonthNo'])['SalesAmount'].sum().reset_index()
budget['Year'] = pd.to_datetime(budget['Date']).dt.year
budget['MonthNo'] = pd.to_datetime(budget['Date']).dt.month
merged = monthly_sales.merge(budget, on=['Year','MonthNo'], how='left')

plt.figure(figsize=(12,6))
plt.fill_between(merged['MonthNo'], merged['SalesAmount'], alpha=0.5, label='Actual Sales')
plt.plot(merged['MonthNo'], merged['Budget'], color='red', linestyle='--', label='Budget')
plt.legend()
plt.title('Monthly Sales vs Budget (Area Chart)')
plt.show()

# Rolling Average Trend
merged['RollingAvg'] = merged['SalesAmount'].rolling(3).mean()
plt.figure(figsize=(12,6))
sns.lineplot(x='MonthNo', y='SalesAmount', data=merged, marker='o', label='Actual Sales')
sns.lineplot(x='MonthNo', y='RollingAvg', data=merged, marker='o', label='3-Month Rolling Avg', color='orange')
plt.title('Sales Trend with Rolling Average')
plt.show()

## 🎯 RFM Segmentation

In [ ]:
snapshot_date = df['OrderDate'].max() + pd.Timedelta(days=1)

rfm = df.groupby('CustomerKey').agg({
    'OrderDate': lambda x: (snapshot_date - x.max()).days,
    'SalesOrderNumber': 'nunique',
    'SalesAmount': 'sum'
}).reset_index()

rfm.rename(columns={'OrderDate':'Recency','SalesOrderNumber':'Frequency','SalesAmount':'Monetary'}, inplace=True)
rfm.head()

# Scatter plot for RFM
plt.figure(figsize=(8,6))
sns.scatterplot(data=rfm, x='Frequency', y='Monetary', size='Monetary', hue='Recency', palette='viridis', alpha=0.7, sizes=(50,500))
plt.title('RFM Segmentation: Frequency vs Monetary (Color=Recency)')
plt.xlabel('Frequency (#Orders)')
plt.ylabel('Monetary (Total Spend)')
plt.show()

## 🔗 Correlation Analysis

In [ ]:
plt.figure(figsize=(6,4))
sns.heatmap(df[['SalesAmount','MonthNo','Year']].corr(), annot=True, cmap='coolwarm')
plt.title('Correlation Heatmap')
plt.show()